<a href="https://colab.research.google.com/github/pramodith/llm_exploration/blob/bert_sparse_attention_training/bert_sparse_attention_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers
%pip install datasets
%pip install accelerate -U
%pip install scikit-learn
%pip install overrides

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from functools import partial
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from datasets import load_dataset, load_metric, load_from_disk
from transformers import TrainingArguments, Trainer
import torch

from transformers import BertModel, BertForSequenceClassification, BertForMaskedLM
from transformers.data.data_collator import _torch_collate_batch
from transformers.models.bert.modeling_bert import BertEncoder, logger
from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions, BaseModelOutputWithPoolingAndCrossAttentions
from transformers.modeling_utils import ModuleUtilsMixin, warnings
from typing import Any, Dict, Optional, Tuple, Union, List, Mapping
from overrides import overrides
from torch import Tensor


In [3]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sample_text = "Neetu Neetu Neetu"
glue_classification_tasks = ["sst2", "cola"]
glue_nli_tasks = ["mnli", "qnli", "rte", "wnli", "ax"]
glue_semantic_similarity_tasks = ["mrpc", "qqp", "stsb"]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def get_dataset(glue_task_name: str):

    # Load the dataset and metric
    dataset = load_dataset('glue', glue_task_name)
    metric = load_metric('glue', glue_task_name)

    # Split the dataset
    train_dataset = dataset['train']
    dev_dataset = dataset['validation']
    test_dataset = dataset['test']

    # Print a description of the dataset
    print("Dataset Description: ", train_dataset.description)

    # Print the label space
    print("Label Space: ", train_dataset.features["label"].names)
    return train_dataset, dev_dataset, test_dataset, metric

In [5]:
train_dataset, dev_dataset, test_dataset, metric = get_dataset('sst2')
train_dataset

/tmp/ipykernel_105/1255836582.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', glue_task_name)


Dataset Description:  GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.


Label Space:  ['negative', 'positive']


Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
class CustomBertEncoder(BertEncoder):
    def __init__(self, config):
        super().__init__(config)

    @overrides
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False

        attention_mask_is_layerwise = False
        if attention_mask.dim() == 5:
            attention_mask_is_layerwise = True

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if attention_mask_is_layerwise:
               attention_mask.shape
               attention_mask_to_use = attention_mask[i]
            else:
                attention_mask_to_use = attention_mask

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    layer_module.__call__,
                    hidden_states,
                    attention_mask_to_use,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask_to_use,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


class CustomBertModel(BertModel):
    def __init__(self, config):
        super().__init__(config)
        self.encoder = CustomBertEncoder(config)

    @overrides
    def get_extended_attention_mask(
        self, attention_mask: Tensor, input_shape: Tuple[int], device: torch.device = None, dtype: torch.float = None
    ) -> Tensor:
        """
        Makes broadcastable attention and causal masks so that future and masked tokens are ignored.

        Arguments:
            attention_mask (`torch.Tensor`):
                Mask with ones indicating tokens to attend to, zeros for tokens to ignore.
            input_shape (`Tuple[int]`):
                The shape of the input to the model.

        Returns:
            `torch.Tensor` The extended attention mask, with a the same dtype as `attention_mask.dtype`.
        """
        if dtype is None:
            dtype = self.dtype

        if not (attention_mask.dim() == 2 and self.config.is_decoder):
            # show warning only if it won't be shown in `create_extended_attention_mask_for_decoder`
            if device is not None:
                warnings.warn(
                    "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
                )
        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        if attention_mask.dim() == 4:
            extended_attention_mask = attention_mask[:, :, None, :, :]
        elif attention_mask.dim() == 3:
            extended_attention_mask = attention_mask[:, None, :, :]
        elif attention_mask.dim() == 2:
            # Provided a padding mask of dimensions [batch_size, seq_length]
            # - if the model is a decoder, apply a causal mask in addition to the padding mask
            # - if the model is an encoder, make the mask broadcastable to [batch_size, num_heads, seq_length, seq_length]
            if self.config.is_decoder:
                extended_attention_mask = ModuleUtilsMixin.create_extended_attention_mask_for_decoder(
                    input_shape, attention_mask, device
                )
            else:
                extended_attention_mask = attention_mask[:, None, None, :]
        else:
            raise ValueError(
                f"Wrong shape for input_ids (shape {input_shape}) or attention_mask (shape {attention_mask.shape})"
            )

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and the dtype's smallest value for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * torch.finfo(dtype).min
        return extended_attention_mask

    @overrides
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.

            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            self.warn_if_padding_and_no_attention_mask(input_ids, attention_mask)
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.bert = CustomBertModel(config)



In [8]:
def torch_mask_tokens(tokenizer: AutoTokenizer, mlm_probability: float, inputs: Any, special_tokens_mask: Optional[Any] = None) -> Tuple[Any, Any]:
    """
    Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
    """
    import torch

    labels = inputs.clone()
    # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    if special_tokens_mask is None:
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
    else:
        special_tokens_mask = special_tokens_mask.bool()

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

In [9]:
from typing import List
import torch
from transformers import AutoTokenizer

def custom_tokenize(tokenizer: AutoTokenizer, text: str, batch_mode=False):
    # Tokenize the texts
    result = tokenizer(text, truncation=True, padding=False)
    # Create attention mask with ones on the main diagonal
    attention_mask = torch.eye(len(result["input_ids"]), dtype=torch.long)

    # Update attention mask for the specified neighborhood distance
    distance = 2
    attention_mask[abs(torch.arange(len(attention_mask))[:, None] - torch.arange(len(attention_mask))) <= distance] = 1

    # Set the first row to 1 corresponding to the CLS token
    attention_mask[0, :] = 1
    # Always attend to CLS
    attention_mask[:, 0] = 1
    # Set the last row to 1 corresponding to the SEP token
    attention_mask[-1, :] = 1
    # Always attend to SEP
    attention_mask[:, -1] = 1
    # Add the attention mask to the result

    if batch_mode:
      result["attention_mask"] = torch.LongTensor(attention_mask.unsqueeze(0))
      result["input_ids"] = torch.LongTensor(result["input_ids"]).unsqueeze(0)
      result["token_type_ids"] = torch.LongTensor(result["token_type_ids"]).unsqueeze(0)
    else:
      result["attention_mask"] = torch.LongTensor(attention_mask)
      result["input_ids"] = torch.LongTensor(result["input_ids"])
      result["token_type_ids"] = torch.LongTensor(result["token_type_ids"])
    # Map the labels to the tokenized inputs
    return result

In [10]:
test_custom_model = CustomBertForSequenceClassification.from_pretrained(model_name)
inputs = custom_tokenize(tokenizer, sample_text, batch_mode=True)
output = test_custom_model(**inputs, output_attentions=True)
print(f"Attention score for Layer 0, Batch 0, Head 0 {output.attentions[0][0,0]}")
print(f"Attention score for Layer 1, Batch 0, Head 0 {output.attentions[1][0,0]}")

inputs["attention_mask"] = inputs["attention_mask"].unsqueeze(0).repeat(12,1,1,1)
output_layerwise_mask = test_custom_model(**inputs, output_attentions=True)
assert torch.allclose(output.attentions[0], output_layerwise_mask.attentions[0]), "Attention scores have to be the same for the same attention mask even if it is repeated layerwise"

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Attention score for Layer 0, Batch 0, Head 0 tensor([[0.0883, 0.0511, 0.0975, 0.0484, 0.1103, 0.0534, 0.1214, 0.4296],
        [0.1157, 0.1826, 0.2381, 0.2117, 0.0000, 0.0000, 0.0000, 0.2519],
        [0.0730, 0.2512, 0.1610, 0.2600, 0.1738, 0.0000, 0.0000, 0.0810],
        [0.0701, 0.1361, 0.1908, 0.1465, 0.1806, 0.1343, 0.0000, 0.1417],
        [0.0654, 0.0000, 0.1433, 0.2158, 0.1446, 0.2198, 0.1364, 0.0748],
        [0.0835, 0.0000, 0.0000, 0.1739, 0.2043, 0.1475, 0.1943, 0.1965],
        [0.1013, 0.0000, 0.0000, 0.0000, 0.2397, 0.3462, 0.2186, 0.0943],
        [0.1801, 0.0651, 0.0971, 0.0677, 0.1166, 0.0731, 0.1155, 0.2849]],
       grad_fn=<SelectBackward0>)
Attention score for Layer 1, Batch 0, Head 0 tensor([[0.5939, 0.0197, 0.0710, 0.0223, 0.0849, 0.0269, 0.0928, 0.0885],
        [0.7525, 0.0096, 0.0368, 0.0065, 0.0000, 0.0000, 0.0000, 0.1946],
        [0.6739, 0.0193, 0.1131, 0.0217, 0.1433, 0.0000, 0.0000, 0.0286],
        [0.7726, 0.0060, 0.0345, 0.0041, 0.0395, 0.0046, 0.00

In [11]:
def get_layerwise_attention_mask(attention_mask: Tensor, pad_token_pos, total_num_layers=12, full_attention_layers=[]):
  attention_mask = attention_mask.unsqueeze(0)
  if full_attention_layers:
    attention_mask = attention_mask.repeat(total_num_layers, 1, 1, 1)
    for layer_num in full_attention_layers:
      attention_mask[layer_num, :, : pad_token_pos, : pad_token_pos] = 1
  return attention_mask

In [12]:
dummy_attn_mask = torch.zeros(1, 4, 4)
layerwise_attn_mask = get_layerwise_attention_mask(dummy_attn_mask, 4, full_attention_layers=[2,4])
assert layerwise_attn_mask.shape == (12, 1, 4, 4), f"{layerwise_attn_mask.shape} is the wrong shape"
assert torch.allclose(layerwise_attn_mask[0], dummy_attn_mask), "Masks at layer 0 should all be 0s"
assert torch.allclose(layerwise_attn_mask[2], torch.ones(1,4,4)), "Masks at layer 1 should all be 1s"

In [13]:
from torch.nn.functional import pad
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch, pad_token_id, full_attention_layers=[], mlm_probability=0.15, is_mlm=False):
  input_ids = [torch.LongTensor(batch[i]["input_ids"]) for i in range(len(batch))]
  attention_mask = [torch.LongTensor(batch[i]["attention_mask"]) for i in range(len(batch))]
  token_type_ids = [torch.LongTensor(batch[i]["token_type_ids"]) for i in range(len(batch))]
  if "label" in batch[0]:
    label = torch.LongTensor([batch[i]["label"] for i in range(len(batch))])
  #idx = [batch[i]["idx"] for i in range(len(batch))]
  max_len = max([len(inp) for inp in input_ids])
  padding_sizes = [max_len - len(inp) for inp in input_ids]
  input_ids = pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
  token_type_ids = pad_sequence(token_type_ids, batch_first=True)
  attention_mask = [get_layerwise_attention_mask(pad(attention_mask[i], (0, padding_sizes[i], 0, padding_sizes[i]), value=0), padding_sizes[i], full_attention_layers=full_attention_layers).squeeze(1) for i in range(len(batch))]
  attention_mask = torch.stack(attention_mask)
  if full_attention_layers:
    attention_mask = attention_mask.permute(1,0,2,3)
  else:
    attention_mask = attention_mask.squeeze(1)

  if is_mlm:
    input_ids, label = torch_mask_tokens(tokenizer, mlm_probability, input_ids)
  return {
      "input_ids": input_ids,
      "attention_mask": attention_mask,
      "token_type_ids": token_type_ids,
      "labels": label,
  }

In [14]:
custom_collate([{"input_ids":[1,2,3,4], "attention_mask":[[1,1,1,0],[1,1,1,0],[1,1,1,0],[1,1,1,0]], "token_type_ids":[0,0,0,0], "label":1}, {"input_ids":[1,2,3,4,5], "attention_mask":[[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0]], "token_type_ids":[0,0,0,0,0], "label":1}], tokenizer.pad_token_id, full_attention_layers=[2,4], is_mlm=True)

{'input_ids': tensor([[1, 2, 3, 4, 0],
         [1, 2, 3, 4, 5]]),
 'attention_mask': tensor([[[[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [0, 0, 0, 0, 0]],
 
          [[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0]]],
 
 
         [[[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [0, 0, 0, 0, 0]],
 
          [[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0]]],
 
 
         [[[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [0, 0, 0, 0, 0]],
 
          [[1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0],
           [1, 1, 1, 0, 0]]],
 
 
         [[[1, 1, 1, 0, 0],
           [1, 1, 1,

In [15]:
a = torch.zeros(8,12,3,3)
a.permute(1,0,2,3).shape

torch.Size([12, 8, 3, 3])

In [16]:
# Encode the datasets
train_dataset_dense_attention = train_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)
dev_dataset_dense_attention = dev_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)
test_dataset_dense_attention = test_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)



train_dataset_sparse_attention = train_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)
dev_dataset_sparse_attention = dev_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)
test_dataset_sparse_attention = test_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [17]:
train_dataset_sparse_attention[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 1],
  [1, 1, 1, 1, 1, 1, 0, 0, 0, 1],
  [1, 0, 1, 1, 1, 1, 1, 0, 0, 1],
  [1, 0, 0, 1, 1, 1, 1, 1, 0, 1],
  [1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,            # total number of training steps
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_ratio=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    fp16=True,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    logging_steps=100
)

# Initialize the trainer
trainer_dense_attention = Trainer(
    model=model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset_dense_attention,         # training dataset
    eval_dataset=dev_dataset_dense_attention,       # evaluation dataset
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics
)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer_dense_attention.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.113900,0.280868,0.915138
200,0.252700,0.227146,0.912844
300,0.226600,0.238781,0.909404
400,0.221600,0.237614,0.912844
500,0.196600,0.262964,0.885321
600,0.183700,0.217739,0.919725
700,0.183600,0.220342,0.917431
800,0.180900,0.201191,0.926606
900,0.168900,0.210163,0.923165
1000,0.157400,0.214784,0.920872


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=3159, training_loss=0.11348400654842598, metrics={'train_runtime': 315.8455, 'train_samples_per_second': 639.702, 'train_steps_per_second': 10.002, 'total_flos': 4633893920893020.0, 'train_loss': 0.11348400654842598, 'epoch': 3.0})

In [ ]:
trainer_dense_attention.evaluate(dev_dataset_dense_attention)

{'eval_loss': 0.21478354930877686,
 'eval_accuracy': 0.9208715596330275,
 'eval_runtime': 0.3051,
 'eval_samples_per_second': 2858.042,
 'eval_steps_per_second': 22.943,
 'epoch': 3.0}

In [47]:
trainer_sparse_attention = Trainer(
    model=model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset_sparse_attention,         # training dataset
    eval_dataset=dev_dataset_sparse_attention,           # evaluation dataset
    data_collator=partial(custom_collate, pad_token_id=tokenizer.pad_token_id),
    compute_metrics=compute_metrics
  )

trainer_sparse_attention.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Accuracy
100,0.670200,0.599721,0.693807
200,0.455000,0.412350,0.815367


KeyboardInterrupt: 

In [ ]:
trainer_sparse_attention.evaluate(dev_dataset_sparse_attention)

Step,Training Loss,Validation Loss,Accuracy
100,0.131900,0.282347,0.893349
167,0.131900,0.298096,0.889908


{'eval_loss': 0.2980963885784149, 'eval_accuracy': 0.8899082568807339}

In [15]:
test_custom_model = CustomBertForSequenceClassification.from_pretrained(model_name)

trainer_sparse_layerwise_attention = Trainer(
    model=test_custom_model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset_sparse_attention,         # training dataset
    eval_dataset=dev_dataset_sparse_attention,           # evaluation dataset
    data_collator=partial(custom_collate, pad_token_id=tokenizer.pad_token_id, full_attention_layers=[0,1,2,3]),
    compute_metrics=compute_metrics
  )

trainer_sparse_layerwise_attention.train()

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: ignored

In [ ]:
trainer_sparse_layerwise_attention.evaluate(dev_dataset_sparse_attention)

{'eval_loss': 0.2847082018852234,
 'eval_accuracy': 0.8990825688073395,
 'eval_runtime': 0.8041,
 'eval_samples_per_second': 1084.416,
 'eval_steps_per_second': 8.705,
 'epoch': 3.0}

In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

### BERT Pre-training with Sparse Attention

In [14]:
def get_pretraining_dataset(dataset_name):
    ds = load_dataset(dataset_name, split='train', streaming=True)
    return ds.take(1280000), ds.take(128000)

In [15]:
from functools import partial
from datasets import Dataset

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

In [16]:
pretraining_train_dataset, pretraining_dev_dataset  = get_pretraining_dataset("Skylion007/openwebtext")

In [17]:
pretraining_train_dataset = Dataset.from_generator(partial(gen_from_iterable_dataset, pretraining_train_dataset), features=pretraining_train_dataset.features)
pretraining_dev_dataset = Dataset.from_generator(partial(gen_from_iterable_dataset, pretraining_dev_dataset), features=pretraining_dev_dataset.features)

In [18]:
pretraining_train_dataset.save_to_disk("./datasets/pretraining_train")
pretraining_dev_dataset.save_to_disk("./datasets/pretraining_dev")

Saving the dataset (0/13 shards):   0%|          | 0/1280000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [19]:
pretraining_train_dataset = load_from_disk("./datasets/pretraining_train")
pretraining_dev_dataset = load_from_disk("./datasets/pretraining_dev")

In [20]:
pretraining_train_dataset

Dataset({
    features: ['text'],
    num_rows: 1181538
})

In [21]:
# pretraining_train_dataset = pretraining_train_dataset.map(lambda example: custom_tokenize(tokenizer, example["text"]), batched=False)
# pretraining_dev_dataset = pretraining_dev_dataset.map(lambda example: custom_tokenize(tokenizer, example["text"]), batched=False)


In [22]:
from torch.utils.data import Dataset

class PretrainingDataset:
    
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return custom_tokenize(self.tokenizer, self.dataset[idx]["text"])

pretraining_train_torch_dataset = PretrainingDataset(pretraining_train_dataset, tokenizer)
pretraining_dev_torch_dataset = PretrainingDataset(pretraining_dev_dataset, tokenizer)

In [23]:
import math
from torch.nn.functional import cross_entropy
def compute_pretraining_metrics(eval_pred):
    logits = torch.from_numpy(eval_pred.predictions)
    labels = torch.from_numpy(eval_pred.label_ids)
    loss = cross_entropy(logits.view(-1, tokenizer.vocab_size), labels.view(-1))
    return {'perplexity': math.exp(loss), 'calculated_loss': loss}


In [25]:
pretraining_args = TrainingArguments(
    output_dir='./bert-sparse-sliding-window-attention',          # output directory
    max_steps=1000,            # total number of training steps
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_ratio=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    fp16=True,
    gradient_checkpointing=False,
    evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200,
    load_best_model_at_end=True,
    logging_steps=200,
    eval_accumulation_steps=5,
    gradient_accumulation_steps=16,
    push_to_hub=True
)

pretrainer_dense_attention = Trainer(
    model=model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=pretraining_args,                  # training arguments, defined above
    train_dataset=pretraining_train_torch_dataset,         # training dataset
    eval_dataset=pretraining_dev_torch_dataset,           # evaluation dataset
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer),
    compute_metrics=compute_pretraining_metrics
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [34]:
#!huggingface-cli login

In [26]:
class CustomBertForMaskedLM(BertForMaskedLM):
  def __init__(self, config):
    super().__init__(config)
    self.bert = CustomBertModel(config)

In [27]:
custom_mlm_model = CustomBertForMaskedLM.from_pretrained(model_name)

In [28]:
pretrainer_sparse_layerwise_attention = Trainer(
    model=custom_mlm_model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=pretraining_args,                  # training arguments, defined above
    train_dataset=pretraining_train_torch_dataset,         # training dataset
    eval_dataset=pretraining_dev_torch_dataset,           # evaluation dataset
    data_collator = partial(custom_collate, pad_token_id=tokenizer.pad_token_id, is_mlm=True),
    compute_metrics=compute_pretraining_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [29]:
print(f"Perplexity before training : {pretrainer_sparse_layerwise_attention.evaluate(pretraining_dev_torch_dataset)}")

KeyboardInterrupt: 

In [30]:
pretrainer_sparse_layerwise_attention.train()

In [39]:
print(f"Perplexity after training : {pretrainer_sparse_layerwise_attention.evaluate(pretraining_dev_dataset)}")

{'eval_loss': 4.437759876251221,
 'eval_perplexity': 84.31074154420784,
 'eval_calculated_loss': 4.43450927734375,
 'eval_runtime': 48.8386,
 'eval_samples_per_second': 4.095,
 'eval_steps_per_second': 0.512,
 'epoch': 6.15}

In [ ]:
!nvidia-smi